### Import libraries



In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

### Model Implementation

In [ ]:
df = pd.read_csv('datasets/preprocessed_train.csv')

# Convert ratings to one-hot encoded labels
labels = to_categorical(df['rating'] + 1)  # Adding 1 to convert -1, 0, 1 to 0, 1, 2

# Tokenize the Arabic text
tokenizer_arabic = Tokenizer()
tokenizer_arabic.fit_on_texts(df['preprocessed_review'])
sequences_arabic = tokenizer_arabic.texts_to_sequences(df['preprocessed_review'])
padded_sequences_arabic = pad_sequences(sequences_arabic)

In [ ]:
# Define the LSTM model with dropout layers
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer_arabic.word_index) + 1, output_dim=100, input_length=padded_sequences_arabic.shape[1]))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) # Adding dropout to the LSTM layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout to the Dense layer
model.add(Dense(3, activation='softmax'))  # Three output nodes for negative, neutral, and positive

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(padded_sequences_arabic, labels, epochs=5, batch_size=32, validation_split=0.2)

In [ ]:
loss, accuracy = model.evaluate(padded_sequences_arabic, labels, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))

### Test Prediction

In [ ]:
test_data = pd.read_csv('datasets/preprocessed_test.csv')

# Tokenize the preprocessed reviews in the test data
sequences_test = tokenizer_arabic.texts_to_sequences(test_data['preprocessed_review'])
padded_sequences_test = pad_sequences(sequences_test, maxlen=padded_sequences_arabic.shape[1])

In [ ]:
predictions = model.predict(padded_sequences_test)
predicted_ratings = np.argmax(predictions, axis=1) - 1

In [ ]:
test_data['rating'] = predicted_ratings
test_data.to_csv('predicted_test.csv', index=False)